In [1]:
import pandas as pd
import numpy as np
from langchain.document_loaders import PyPDFLoader 
import re
from langchain_huggingface import HuggingFaceEmbeddings
import glob
import os

In [2]:
def cosine_similarity(A, B):
    return np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))

In [3]:
1

1

In [4]:
alphabet = "abcdefghijklmnopqrstuvwxyz"

# Load Classes Descriptions

In [5]:
df = pd.read_csv("NACE_Rev2_Structure_Explanatory_Notes_EN__1_.tsv", sep="\t")
df

,ORDER_KEY,ID,CODE,NAME,PARENT_ID,LEVEL,Includes,IncludesAlso,Excludes
0,200,A,A,"AGRICULTURE, FORESTRY AND FISHING",NaN,1,This section includes the exploitation of vege...,NaN,NaN
1,300,01,01,"Crop and animal production, hunting and relate...",A,2,"This division includes two basic activities, n...",This division also includes service activities...,Agricultural activities exclude any subsequent...
2,350,011,01.1,Growing of non-perennial crops,01,3,This group includes the growing of non-perenni...,NaN,NaN
3,355,0111,01.11,"Growing of cereals (except rice), leguminous c...",011,4,This class includes all forms of growing of ce...,NaN,"This class excludes:\n- growing of rice, see 0..."
4,360,0112,01.12,Growing of rice,011,4,This class includes:\n- growing of rice (inclu...,NaN,NaN
...,...,...,...,...,...,...,...,...,...
991,51295,9820,98.20,Undifferentiated service-producing activities ...,982,4,This class includes the undifferentiated subsi...,NaN,NaN
992,51495,U,U,ACTIVITIES OF EXTRATERRITORIAL ORGANISATIONS A...,NaN,1,NaN,NaN,NaN
993,51595,99,99,Activities of extraterritorial organisations a...,U,2,NaN,NaN,NaN
994,51645,990,99.0,Activities of extraterritorial organisations a...,99,3,NaN,NaN,NaN


In [6]:
df_first_level = df[df["ID"].apply(lambda x: not x.isnumeric())]
df_first_level = df_first_level.dropna(subset=["Includes"])
df_first_level.reset_index(drop=True, inplace=True)
df_first_level

,ORDER_KEY,ID,CODE,NAME,PARENT_ID,LEVEL,Includes,IncludesAlso,Excludes
0,200,A,A,"AGRICULTURE, FORESTRY AND FISHING",NaN,1,This section includes the exploitation of vege...,NaN,NaN
1,1945,B,B,MINING AND QUARRYING,NaN,1,Mining and quarrying include the extraction of...,NaN,This section excludes:\n- processing of the ex...
2,3220,C,C,MANUFACTURING,NaN,1,This section includes the physical or chemical...,NaN,NaN
3,15650,D,D,"ELECTRICITY, GAS, STEAM AND AIR CONDITIONING S...",NaN,1,This section includes the activity of providin...,Also included is the provision of steam and ai...,This section excludes the operation of water a...
4,16260,E,E,"WATER SUPPLY; SEWERAGE, WASTE MANAGEMENT AND R...",NaN,1,This section includes activities related to th...,Activities of water supply are also grouped in...,NaN
5,17650,F,F,CONSTRUCTION,NaN,1,This section includes general construction and...,This section also includes the development of ...,If these activities are carried out not for la...
6,21175,G,G,WHOLESALE AND RETAIL TRADE; REPAIR OF MOTOR VE...,NaN,1,This section includes wholesale and retail sal...,NaN,NaN
7,25630,H,H,TRANSPORTATION AND STORAGE,NaN,1,This section includes the provision of passeng...,NaN,This section excludes:\n- major repair or alte...
8,28260,I,I,ACCOMMODATION AND FOOD SERVICE ACTIVITIES,NaN,1,This section includes the provision of short-s...,NaN,This section excludes the provision of long-te...
9,29055,J,J,INFORMATION AND COMMUNICATION,NaN,1,This section includes the production and distr...,NaN,NaN


# Load Reports

In [7]:
pdf_path = "annual_reports/mercedes-benz-annual-report-2023-incl-combined-management-report-mbg-ag-2.pdf"

In [8]:
loader = PyPDFLoader(pdf_path)
seiten_docs = loader.load()

In [9]:
seiten_docs[0].page_content

'Annual Report 2023'

In [10]:
text = " ".join([page.page_content for page in seiten_docs])

In [11]:
sentences = text.split(".")

In [12]:
sentences = [sentence.lower().strip() for sentence in sentences]
sentences = [re.sub(r"[^a-zA-ZäöüÄÖÜß\s]", "", sentence) for sentence in sentences]

In [13]:
sentences

['annual report  contents  to our shareholders combined management report corporate governance consolidated financial statements further information \n   \nannual report       mercedesbenz group\ncontents to our shareholders combined management report corporate governance  consolidated financial statements further information \n   \nannual report       mercedesbenz group\n  to our shareholders\n  combined management report\n  corporate governance \n  consolidated financial statements\n  further information  to our \nshareholders  contents  to our shareholders combined management report corporate governance consolidated financial statements further information \n   \nannual report       mercedesbenz group\nto our shareholders\n  letter from the ceo \n  the board of management\n  report of the supervisory board\n  the supervisory board\n  objectives and strategy\n  mercedesbenz cars strategy\n  mercedesbenz vans strategy\n  mercedesbenz mobility strategy contents  to our shareholders com

In [14]:
embedding_model = HuggingFaceEmbeddings(
                model_name="sentence-transformers/all-mpnet-base-v2",
                #model_kwargs=model_kwargs,
                #encode_kwargs=encode_kwargs
            )

### Embed sentences

In [15]:
df_sentences = pd.DataFrame(sentences, columns=["Sentences"])
df_sentences["Embeddings"] = embedding_model.embed_documents(df_sentences["Sentences"].to_list())


In [16]:
df_sentences


,Sentences,Embeddings
0,annual report contents to our shareholders c...,"[0.008581217378377914, -0.013776358217000961, ..."
1,we were \nable to assert ourselves strongly in...,"[0.0684594064950943, 0.02481973171234131, -0.0..."
2,this is also reflected in our balance sheet,"[-0.010797382332384586, -0.04653540998697281, ..."
3,we also \ncontinued to make progress in the i...,"[0.05377557501196861, 0.06135500594973564, -0...."
4,our company has shown that we can achieve soli...,"[-0.016974974423646927, 0.054922617971897125, ..."
...,...,...
6788,if any of \nthese risks and uncertainties mate...,"[-0.01739267073571682, -0.006869446486234665, ..."
6789,we do not intend or assume any \nobligation to...,"[0.029017524793744087, 0.07583380490541458, -0..."
6790,statements regarding electricity and fuel cons...,"[-0.0012728744186460972, -0.039766207337379456..."
6791,defileadminmedia\nleitfadenco guideco,"[0.0383455865085125, -0.05001586303114891, -0...."


### Embed classes

In [17]:
df_first_level["Embeddings"] = embedding_model.embed_documents(df_first_level["Includes"].to_list())

### Caclulate Similarities

In [18]:
for i, row in df_first_level.iterrows(): 
    similarities = df_sentences["Embeddings"].apply(lambda x: cosine_similarity(x, row.Embeddings))
    df_sentences[f"Scores_{alphabet[i]}_{row.NAME}"] = similarities

In [19]:
df_sentences

,Sentences,Embeddings,"Scores_a_AGRICULTURE, FORESTRY AND FISHING",Scores_b_MINING AND QUARRYING,Scores_c_MANUFACTURING,"Scores_d_ELECTRICITY, GAS, STEAM AND AIR CONDITIONING SUPPLY","Scores_e_WATER SUPPLY; SEWERAGE, WASTE MANAGEMENT AND REMEDIATION ACTIVITIES",Scores_f_CONSTRUCTION,Scores_g_WHOLESALE AND RETAIL TRADE; REPAIR OF MOTOR VEHICLES AND MOTORCYCLES,Scores_h_TRANSPORTATION AND STORAGE,...,Scores_j_INFORMATION AND COMMUNICATION,Scores_k_FINANCIAL AND INSURANCE ACTIVITIES,Scores_l_REAL ESTATE ACTIVITIES,"Scores_m_PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES",Scores_n_ADMINISTRATIVE AND SUPPORT SERVICE ACTIVITIES,Scores_o_PUBLIC ADMINISTRATION AND DEFENCE; COMPULSORY SOCIAL SECURITY,Scores_p_EDUCATION,Scores_q_HUMAN HEALTH AND SOCIAL WORK ACTIVITIES,"Scores_r_ARTS, ENTERTAINMENT AND RECREATION",Scores_s_OTHER SERVICE ACTIVITIES
0,annual report contents to our shareholders c...,"[0.008581217378377914, -0.013776358217000961, ...",0.063642,0.229635,0.236262,0.163826,0.165014,0.178897,0.319222,0.281765,...,0.153778,0.259213,0.088421,0.147318,0.244572,0.037413,0.118207,0.028264,0.081005,0.207571
1,we were \nable to assert ourselves strongly in...,"[0.0684594064950943, 0.02481973171234131, -0.0...",0.049305,0.024133,-0.016023,0.066233,0.021023,0.024876,-0.000426,0.018060,...,0.054937,0.017866,-0.018747,0.102038,0.031466,0.103008,0.034571,0.061611,0.022055,0.062010
2,this is also reflected in our balance sheet,"[-0.010797382332384586, -0.04653540998697281, ...",0.101771,0.136009,0.132018,0.141665,0.138731,0.174488,0.184317,0.230473,...,0.172881,0.348618,0.174029,0.181921,0.233965,0.147559,0.116147,0.079622,0.106152,0.226155
3,we also \ncontinued to make progress in the i...,"[0.05377557501196861, 0.06135500594973564, -0....",0.112611,0.210927,0.193345,0.149975,0.186254,0.210583,0.162793,0.176970,...,0.150984,0.195899,0.036856,0.211694,0.216378,0.152931,0.124016,0.171535,0.083697,0.163736
4,our company has shown that we can achieve soli...,"[-0.016974974423646927, 0.054922617971897125, ...",0.134051,0.176804,0.121902,0.149520,0.176926,0.117699,0.152137,0.188552,...,0.105369,0.296415,0.061248,0.208226,0.179846,0.079286,0.038028,0.065696,0.099382,0.185800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6788,if any of \nthese risks and uncertainties mate...,"[-0.01739267073571682, -0.006869446486234665, ...",0.092514,0.157989,0.199013,0.132795,0.168801,0.235629,0.177320,0.124439,...,0.084555,0.186693,0.148075,0.252872,0.248873,0.144759,0.130435,0.127341,0.055713,0.177744
6789,we do not intend or assume any \nobligation to...,"[0.029017524793744087, 0.07583380490541458, -0...",0.017308,0.128567,0.193168,0.128071,0.147971,0.132188,0.244962,0.172363,...,0.202475,0.213832,0.154363,0.171685,0.270682,0.170589,0.116486,0.116379,0.008649,0.218341
6790,statements regarding electricity and fuel cons...,"[-0.0012728744186460972, -0.039766207337379456...",0.120929,0.278988,0.287783,0.309364,0.186581,0.185051,0.237034,0.226355,...,0.192600,0.082228,0.031332,0.142441,0.092816,0.114133,0.194238,0.029638,0.022113,0.114227
6791,defileadminmedia\nleitfadenco guideco,"[0.0383455865085125, -0.05001586303114891, -0....",0.161987,0.258999,0.184710,0.187320,0.207139,0.249621,0.266522,0.261799,...,0.281511,0.168616,0.217430,0.323955,0.198766,0.167431,0.237662,0.116734,0.111510,0.183010


# Create Function and generate data for multiple reports!

In [20]:
def create_sentence_nace_code_similarities(pdf_path): 
    # Load Classes Descriptions
    df = pd.read_csv("NACE_Rev2_Structure_Explanatory_Notes_EN__1_.tsv", sep="\t")

    df_first_level = df[df["ID"].apply(lambda x: not x.isnumeric())]
    df_first_level = df_first_level.dropna(subset=["Includes"])
    df_first_level.reset_index(drop=True, inplace=True)

    # Load Reports
    loader = PyPDFLoader(pdf_path)
    seiten_docs = loader.load()
    seiten_docs[0].page_content
    text = " ".join([page.page_content for page in seiten_docs])
    sentences = text.split(".")
    sentences = [sentence.lower().strip() for sentence in sentences]
    sentences = [re.sub(r"[^a-zA-ZäöüÄÖÜß\s]", "", sentence) for sentence in sentences]

    embedding_model = HuggingFaceEmbeddings(
                    model_name="sentence-transformers/all-mpnet-base-v2",
                    #model_kwargs=model_kwargs,
                    #encode_kwargs=encode_kwargs
                )
    
    ### Embed sentences
    df_sentences = pd.DataFrame(sentences, columns=["Sentences"])
    df_sentences["Embeddings"] = embedding_model.embed_documents(df_sentences["Sentences"].to_list())

    df_sentences

    ### Embed classes
    df_first_level["Embeddings"] = embedding_model.embed_documents(df_first_level["Includes"].to_list())
    ### Caclulate Similarities
    for i, row in df_first_level.iterrows(): 
        similarities = df_sentences["Embeddings"].apply(lambda x: cosine_similarity(x, row.Embeddings))
        df_sentences[f"Scores_{alphabet[i]}_{row.NAME}"] = similarities
    return df_sentences

In [21]:
pdf_paths = glob.glob("annual_reports/*.pdf")
pdf_paths

['annual_reports/Deutsche_Annual-Report-2023.pdf',
 'annual_reports/bayer-annual-report-2023-2.pdf',
 'annual_reports/adidas-ar23.pdf',
 'annual_reports/mercedes-benz-annual-report-2023-incl-combined-management-report-mbg-ag-2.pdf',
 'annual_reports/Siemens-Annual-Report-2023.pdf',
 'annual_reports/thyssenkrupp-GB_2023-2024_EN_WEB.pdf',
 'annual_reports/Zalando-SE_DE_241203_s.pdf',
 'annual_reports/heidelberg-materials_2023.pdf',
 'annual_reports/rheinmetall-ag_2023.pdf',
 'annual_reports/conti_annual-report-2023-data.pdf']

In [22]:
df = create_sentence_nace_code_similarities(pdf_path)

In [23]:
for pdf_path in pdf_paths: 
    df = create_sentence_nace_code_similarities(pdf_path)
    df.to_csv(f"sentence_similarities/{os.path.basename(pdf_path).split('.')[0]}.csv")